# Trend Analysis: Seasonality and trend in summer season temperature for Great African Lakes and test for significance

## Import packages

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pymannkendall as mk
from c3s_eqc_automatic_quality_control import diagnostics, download, plot, utils

plt.style.use("seaborn-v0_8-notebook")

## Set varibales

In [ ]:
# Time
start = "1997-01"
stop = "1998-01"

# Region
lon_slice = slice(28, 41)
lat_slice = slice(-16, 4)

# Variable
varname = "lake_surface_water_temperature"

## Set the data request

In [ ]:
collection_id = "satellite-lake-water-temperature"
request = {
    "version": "4.0",
    "variable": "all",
    "format": "zip",
}

## Define function to extract region and compute spatial weighted mean

In [ ]:
def spatial_weighted_mean_of_region(ds, lon_slice, lat_slice, varname):
    ds = ds[[varname]]
    ds = utils.regionalise(ds, lon_slice=lon_slice, lat_slice=lat_slice)
    ds = diagnostics.spatial_weighted_mean(ds)
    return ds

## Download data

In [ ]:
chunks = {"year": 1, "month": 1}
requests = download.update_request_date(
    request, start=start, stop=stop, stringify_dates=True
)
ds = download.download_and_transform(
    collection_id,
    requests,
    chunks=chunks,
    transform_func=spatial_weighted_mean_of_region,
    transform_func_kwargs={
        "lon_slice": lon_slice,
        "lat_slice": lat_slice,
        "varname": varname,
    },
)
da = ds[varname]

## Extract lake id to plot a map of the region

In [ ]:
# We use one of the request previously cached
single_request = requests[0]
single_request["month"] = single_request["month"][0]
ds_raw = download.download_and_transform(
    collection_id,
    single_request,
    chunks=chunks,
)

da_lakeid = utils.regionalise(
    ds_raw["lakeid"].isel(time=0), lon_slice=lon_slice, lat_slice=lat_slice
)

## Plot projected map

In [ ]:
plot.projected_map(da_lakeid, projection=ccrs.PlateCarree())

## Plot spatial weighted mean

In [ ]:
da.plot()
plt.title("Spatial weighted mean")

## Mothly resample and plot

In [ ]:
da.resample(time="1MS").mean().plot()
plt.title("Monthly mean")

## Seasonal boxplot

In [ ]:
plot.seasonal_boxplot(da)
plt.suptitle("Seasonal boxplot")

## Mann-Kendall trend test

In [ ]:
trend, h, p, z, tau, s, var_s, slope, intercept = mk.original_test(da)
print(
    f"The trend of {da.attrs.get('long_name', da.name)} {'is' if p<0.05 else 'is NOT'} significant."
)
print(f"Trend: {slope}")